# Data Transformation

### Import required libraries

In [2]:
from tqdm import tqdm
# import os
# import json
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
# import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np

2023-04-19 15:14:11.721489: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 15:14:12.423968: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-19 15:14:12.424994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 15:14:14.690279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sylvas/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sylvas/anacon

In [3]:
 # Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nltk, time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
import collections, itertools

In [137]:
# Download these NLTK packages
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/sylvas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sylvas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sylvas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/sylvas/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /home/sylvas/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /home/sylvas/nltk_data...


True

### Load dataset

In [10]:
data = pd.read_csv('../datasets/sherloc/sherloc_court_cases_7.csv')

In [11]:
data.head()

,text,crime_types,sentence
0,Mr. Solomon Sauls ran an illegal enterprise wi...,money laundry,1
1,SummaryHarmony Gold Mine (Pty) Limited is a mi...,money laundry,1
2,SummaryThe three defendants were found guilty ...,money laundry,1
3,Johannes Erasmus van Staden was a Cape Town bu...,money laundry,1
4,Juan Hattingh was a young practicing attorney ...,money laundry,1


## Exploration of data

## Transformation

#### Step 1: Sentence segmentation

In [22]:
doc = nltk.sent_tokenize(data.text[0])
#to print sentences
# doc
for sent in doc:
    print(sent)

['Mr. Solomon Sauls ran an illegal enterprise with the purpose of poaching and selling abalone (Haliotis midae) in South Africa.',
 'A group of abalone divers worked under him supplying him with illegally harvested abalone.',
 'The accused bribed officials from the Department of Agriculture, Forestry and Fisheries to prevent them from confiscating abalone and to buy back abalone already seized by the authorities.',
 'The corrupt officials are facing charges in a separate trial.In March 2018, the police searched the accused’s house and encountered cash that the accused confessed to proceeds of his illegal activities.',
 'In February 2021, the defendant pleaded guilty to 41 counts involving running an illegal enterprise, corruption, money laundering and possessing and transporting illegally harvested abalone.Factors aggravating the sentence were the seriousness of corrupting government officials, the engagement in the illegal abalone trade on a commercial scale and financial greed.The ac

#### Step 2: Word tokenization.
- drop duplicate and nan if any
- tokenize text
- transform text into lower case
- remove punctuatios
- remove stopword (step 5)

In [14]:
# Function to tokenize the tweets
def custom_tokenize(text):
    """Function that tokenizes text"""
    from nltk.tokenize import word_tokenize
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)

In [47]:
def clean_up(data):
    """Function that cleans up the data into a shape that can be further used for modeling"""
    # english = data[data['lang']=='en'] # extract only tweets in english language
    data.drop_duplicates() # drop duplicate tweets
    data['text'].dropna(inplace=True) # drop any rows with missing tweets
    tokenized = data['text'].apply(custom_tokenize) # Tokenize tweets
    lower_tokens = tokenized.apply(lambda x: [t.lower() for t in x]) # Convert tokens into lower case
    alpha_only = lower_tokens.apply(lambda x: [t for t in x if t.isalpha()]) # Remove punctuations
    no_stops = alpha_only.apply(lambda x: [t for t in x if t not in stopwords.words('english')]) # remove stop words
    # no_stops.apply(lambda x: [x.remove(t) for t in x if t=='rt']) # remove acronym "rt"
    return no_stops

In [62]:
cleaned_text_chunk = clean_up(data)

In [63]:
cleaned_text_chunk.shape

(1250,)

### Exploratory Data Analysis


### Create bag of words with gensim

In [78]:
# Import Dictionary
from gensim.corpora.dictionary import Dictionary
# Create a Dictionary from the tweets
start = time.time()

dictionary = Dictionary(cleaned_text_chunk)

end = time.time()

total = end-start
total

0.3023958206176758

In [91]:
# save your dictionary as text file
from gensim.test.utils import get_tmpfile
tmp_fname = get_tmpfile("dictionary")
dictionary.save_as_text(tmp_fname)
 
# load your dictionary text file
load_dict = corpora.Dictionary.load_from_text(tmp_fname)

In [92]:
tmp_fname

'/tmp/tmpv0mr26sg/dictionary'

In [94]:
# Create corpus for bag of words (token IDs of each word with their frequencies)
corpus = cleaned_text_chunk.apply(lambda x: dictionary.doc2bow(x))

0.21866464614868164

In [95]:
corpus

0       [(0, 8), (1, 4), (2, 1), (3, 1), (4, 1), (5, 1...
1       [(3, 2), (6, 2), (12, 1), (15, 1), (32, 3), (3...
2       [(0, 6), (1, 1), (2, 1), (3, 1), (7, 4), (23, ...
3       [(1, 1), (12, 2), (15, 2), (23, 1), (29, 2), (...
4       [(15, 2), (23, 1), (31, 1), (44, 2), (51, 3), ...
                              ...                        
1245    [(1, 3), (26, 2), (43, 1), (58, 1), (70, 1), (...
1246    [(235, 1), (346, 1), (791, 1), (1521, 1), (176...
1247    [(1, 1), (26, 1), (48, 1), (58, 1), (70, 1), (...
1248    [(58, 1), (124, 1), (143, 1), (195, 1), (252, ...
1249    [(1, 1), (26, 3), (27, 1), (37, 3), (43, 2), (...
Name: text, Length: 1250, dtype: object

### Renaming columns

In [125]:
data.rename(columns={'text':'raw_text'}, inplace=True)
data.head(2)

AttributeError: 'NoneType' object has no attribute 'head'

In [127]:
df = pd.concat([data,cleaned_text_chunk],axis=1)
df.head(2)

,raw_text,crime_types,sentence,text
0,Mr. Solomon Sauls ran an illegal enterprise wi...,money laundry,1,"[solomon, sauls, ran, illegal, enterprise, pur..."
1,SummaryHarmony Gold Mine (Pty) Limited is a mi...,money laundry,1,"[summaryharmony, gold, mine, pty, limited, min..."


In [131]:
df.rename(columns={'text':'tokenized_cleaned_text'}, inplace=True)
df.head(2)


## Step 4: Lemmatization.

In [132]:
from nltk.corpus import wordnet

In [133]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [138]:
# Lemmatize tokens
start = time.time()

lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['tokenized_cleaned_text'].apply(lambda x: [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in x])

end = time.time()
total = end-start
total

35.417070150375366

In [144]:
df.head()

,raw_text,crime_types,sentence,tokenized_cleaned_text,lemmatized
0,Mr. Solomon Sauls ran an illegal enterprise wi...,money laundry,1,"[solomon, sauls, ran, illegal, enterprise, pur...","[solomon, saul, ran, illegal, enterprise, purp..."
1,SummaryHarmony Gold Mine (Pty) Limited is a mi...,money laundry,1,"[summaryharmony, gold, mine, pty, limited, min...","[summaryharmony, gold, mine, pty, limited, min..."
2,SummaryThe three defendants were found guilty ...,money laundry,1,"[summarythe, three, defendants, found, guilty,...","[summarythe, three, defendant, found, guilty, ..."
3,Johannes Erasmus van Staden was a Cape Town bu...,money laundry,1,"[johannes, erasmus, van, staden, cape, town, b...","[johannes, erasmus, van, staden, cape, town, b..."
4,Juan Hattingh was a young practicing attorney ...,money laundry,1,"[juan, hattingh, young, practicing, attorney, ...","[juan, hattingh, young, practice, attorney, co..."


In [152]:
df.to_csv('../datasets/sherloc/sherloc_court_cases_lemmatized.csv')

In [ ]:
dff = df

In [9]:
# lemmatized text back to tokenized
import ast
# m= ast.literal_eval(f) # convert string to list just like json.loads()
dff['tokens_back_to_text'] = [' '.join(map(str, ast.literal_eval(l))) for l in dff['lemmatized']]
dff.head(2)

TypeError: 'int' object is not iterable

In [34]:
dff.to_csv('../datasets/sherloc/token_back_to_text.csv', index=False)